Purpose of this notebook is to demonstrate dynamic programming as a method for policy approximation.

Aim is for the student to understand
- concept of what we need to define a Markov Decision Process (state transitons, reward transitions)
- how we use the Bellman equation to bootstrap our value function approximation

- how dynamic programming can learn value functions without taking actions]
- how dynamic programming needs the environment model to work


In [2]:
import numpy as np

In [3]:
state_space = np.array(['s{}'.format(state) for state in np.arange(1,6)])
action_space = np.array(['left', 'right', 'up', 'down'])

In [4]:
#  loading state transition probabilities from csvs
state_transitions = {state:np.genfromtxt('{}.csv'.format(state), delimiter=',') for state in state_space}

In [5]:
#  here for example are the state transition probabilities
#  note that rows = actions, columns = states

#  here are the probabilities for state s1
#  the probability of action 'right' taking us to 's2' is 1
#  the probability of action 'up' taking us to 's1' is 1
#  i.e. P(s'|s,a)
state_transitions['s1']

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [24]:
#  we can now do our reward transition probabilities
reward_functions = {state:np.full((len(state_space)),-1) for state in state_space}

#  these work in a similar way to our state transitions, except they are not conditioned upon actions
#  only conditions upon state, next_state
#  i.e R(s, s')
#  this is OK for our problem - a more formal problem would have R(s, a, s')

#  reward of all state->next_state transitons is -1
reward_functions['s4']

array([-1, -1, -1, -1, -1])

In [8]:
#  now we manually fill in the rewards for our special states
reward_functions['s4'][4] = 10
reward_functions['s4']

array([-1, -1, -1, -1, 10])

In [9]:
#  's5' is a special absorbing state with no rewards and transition prob of 1 to itself
reward_functions['s5'] = np.full((len(state_space)),0)
reward_functions['s5']

array([0, 0, 0, 0, 0])

In [10]:
#  we need a policy to approximate - lets use a random policy

def random_policy(state, action_space):
    action = np.random.choice(action_space)
    p_distribution = np.full(len(action_space),  1/len(action_space))
    return action, p_distribution

In [11]:
class Dynamic_Programmer(object):
    def __init__(self, 
                 state_space, 
                 action_space, 
                 state_transitions, 
                 reward_functions,
                 policy,
                 discount,
                 verbose=1):    
        
        self.state_space = state_space
        self.action_space = action_space
        self.state_transitions = state_transitions
        self.reward_functions = reward_functions
        self.policy = policy
        self.discount = discount
        self.verbose = verbose
    
        self.v_table = np.full(len(state_space),0.0)

    def policy_transitions(self, state):
        """
        For a given state, returns a distribution over
        next states (not over actions!) for a given policy
        """
        state_idx = np.argwhere(state==self.state_space).flatten()
        #  our policy gives us a distribution over actions
        _, action_p_dist = self.policy(state, self.action_space)
        
        #  we can now use our perfect environment model to 
        #  develop the state transition probabilities
        
        for next_state in self.state_space:    
            all_probs = np.full(len(state_space),0)
            all_rewards = np.full(len(state_space),0)
            for act_idx, act_prob in enumerate(action_p_dist):   
                
                state_probs = self.state_transitions[state][act_idx].flatten()
                probs = act_prob * state_probs
                all_probs = np.add(all_probs, probs)
                
        return all_probs
    
    def backup_state(self, state, probabilities): 
        rewards = self.reward_functions[state]
        discounted_values = self.discount * self.v_table  # note the recursive use of v.table!
        bellman_equations = np.add(rewards, discounted_values)
        
        state_value = np.sum(np.multiply(probabilities, bellman_equations))
        
        state_idx = np.argwhere(state==self.state_space).flatten()
        self.v_table[state_idx] = state_value
        
        if self.verbose == 1:
            print('backing up state {}'.format(state))
            print('probabilities are {}'.format(probabilities))
            print('rewards are {}'.format(rewards))
            print('discounted_values are {}'.format(discounted_values))  
            print('bellman equations are {}'.format(bellman_equations))
            print('state value is {}'.format(state_value))
            print('value function table {}'.format(self.v_table))
        return state_value
    
    def backup_all_states(self):
        
        for state in self.state_space:
            probabilities = self.policy_transitions(state)
            state_value = self.backup_state(state, probabilities)
        return self.v_table  
            

In [22]:
random_policy_approx = Dynamic_Programmer(state_space, 
                        action_space, 
                        state_transitions,
                        reward_functions,
                        random_policy,
                        discount=0.9,
                        verbose=0)
BACKUPS = 60
for backup in range(BACKUPS):
    value_function = random_policy_approx.backup_all_states()
    print('value function is {}'.format(value_function))

value function is [-1.      -1.225   -1.225    1.19875  0.     ]
value function is [-2.00125    -1.7318125  -1.18646875  1.36310547  0.        ]
value function is [-2.55717578 -2.04798145 -1.22892256  1.31939533  0.        ]
value function is [-2.8880325  -2.27453501 -1.33258699  1.2352615   0.        ]
value function is [-3.11121708 -2.44563076 -1.44398824  1.15276956  0.        ]
value function is [-3.27521196 -2.57808338 -1.54307374  1.0821128   0.        ]
value function is [-3.40110574 -2.68191093 -1.62548962  1.02431025  0.        ]
value function is [-3.49966271 -2.76381422 -1.69221966  0.97786218  0.        ]
value function is [-3.57745584 -2.82862497 -1.745639    0.9408096   0.        ]
value function is [-3.63906452 -2.8799886  -1.78819398  0.91134108  0.        ]
value function is [-3.68792011 -2.92072515 -1.82202218  0.88793359  0.        ]
value function is [-3.7266822  -2.95304475 -1.84888837  0.86935011  0.        ]
value function is [-3.75744194 -2.9786908  -1.87021677 

In [14]:
#  we can now try to evaluate an optimal policy
#  for this simple MDP we can hard code the optimal policy

def optimal_policy(state, action_space):
    
    optimal_action = {'s1' : 'right',
                      's2' : 'down',
                      's3' : 'right',
                      's4' : 'right',
                      's5' : 'right'}

    action = optimal_action[state]

    p_distribution = np.full(len(action_space), 0)
    act_idx = np.argwhere(action == action_space)
    p_distribution[act_idx] = 1
    return action, p_distribution

In [23]:
optimal_policy_approx = Dynamic_Programmer(state_space, 
                        action_space, 
                        state_transitions,
                        reward_functions,
                        optimal_policy,
                        discount=0.9,
                        verbose=0)

BACKUPS = 10
for backup in range(BACKUPS):
    optimal_value_function = optimal_policy_approx.backup_all_states()
    print('value function is {}'.format(optimal_value_function))

value function is [ -1.  -1.  -1.  10.   0.]
value function is [ -1.9   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
value function is [  6.2   8.    8.   10.    0. ]
